In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import glob 
from sklearn.metrics import confusion_matrix
import IPython.display as ipd  # To play sound in the notebook
import os
import sys
import warnings
# ignore warnings 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
RAV = "/kaggle/input/ravdess-emotional-speech-audio/"
SAVEE = "/kaggle/input/surrey-audiovisual-expressed-emotion-savee/ALL/"
TESS = "/kaggle/input/toronto-emotional-speech-set-tess/TESS Toronto emotional speech set data/"
CREMA = "/kaggle/input/cremad/AudioWAV/"

In [3]:
dir_list = os.listdir(TESS)
dir_list.sort()
dir_list

['OAF_Fear',
 'OAF_Pleasant_surprise',
 'OAF_Sad',
 'OAF_angry',
 'OAF_disgust',
 'OAF_happy',
 'OAF_neutral',
 'YAF_angry',
 'YAF_disgust',
 'YAF_fear',
 'YAF_happy',
 'YAF_neutral',
 'YAF_pleasant_surprised',
 'YAF_sad']

In [4]:
path = []
emotion = []

for i in dir_list:
    fname = os.listdir(TESS + i)
    for f in fname:
        if i == 'OAF_angry' or i == 'YAF_angry':
            emotion.append('angry')
        elif i == 'OAF_disgust' or i == 'YAF_disgust':
            emotion.append('disgust')
        elif i == 'OAF_Fear' or i == 'YAF_fear':
            emotion.append('fear')
        elif i == 'OAF_happy' or i == 'YAF_happy':
            emotion.append('happy')
        elif i == 'OAF_neutral' or i == 'YAF_neutral':
            emotion.append('neutral')                                
        elif i == 'OAF_Pleasant_surprise' or i == 'YAF_pleasant_surprised':
            emotion.append('surprise')               
        elif i == 'OAF_Sad' or i == 'YAF_sad':
            emotion.append('sad')
        else:
            emotion.append('Unknown')
        path.append(TESS + i + "/" + f)

TESS_df = pd.DataFrame(emotion, columns = ['labels'])
TESS_df['source'] = 'TESS'
TESS_df = pd.concat([TESS_df,pd.DataFrame(path, columns = ['path'])],axis=1)
TESS_df.labels.value_counts()

angry       400
surprise    400
happy       400
sad         400
disgust     400
neutral     400
fear        400
Name: labels, dtype: int64

In [5]:
TESS_df.head(7)

,labels,source,path
0,fear,TESS,/kaggle/input/toronto-emotional-speech-set-tes...
1,fear,TESS,/kaggle/input/toronto-emotional-speech-set-tes...
2,fear,TESS,/kaggle/input/toronto-emotional-speech-set-tes...
3,fear,TESS,/kaggle/input/toronto-emotional-speech-set-tes...
4,fear,TESS,/kaggle/input/toronto-emotional-speech-set-tes...
5,fear,TESS,/kaggle/input/toronto-emotional-speech-set-tes...
6,fear,TESS,/kaggle/input/toronto-emotional-speech-set-tes...


In [6]:
crema_directory_list = os.listdir(CREMA)

file_emotion = []
file_path = []
source = []

for file in crema_directory_list:
    # storing file paths
    file_path.append(CREMA + file)
    # storing file emotions
    source.append("CREMA")
    part=file.split('_')
    if part[2] == 'SAD':
        file_emotion.append('sad')
    elif part[2] == 'ANG':
        file_emotion.append('angry')
    elif part[2] == 'DIS':
        file_emotion.append('disgust')
    elif part[2] == 'FEA':
        file_emotion.append('fear')
    elif part[2] == 'HAP':
        file_emotion.append('happy')
    elif part[2] == 'NEU':
        file_emotion.append('neutral')
    else:
        file_emotion.append('Unknown')
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['labels'])
#
source_df = pd.DataFrame(source,columns=['source'])
# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['path'])
Crema_df = pd.concat([emotion_df,source_df,path_df], axis=1)
Crema_df.labels.value_counts()

angry      1271
happy      1271
disgust    1271
fear       1271
sad        1271
neutral    1087
Name: labels, dtype: int64

In [7]:
Crema_df.head()

,labels,source,path
0,disgust,CREMA,/kaggle/input/cremad/AudioWAV/1028_TSI_DIS_XX.wav
1,happy,CREMA,/kaggle/input/cremad/AudioWAV/1075_IEO_HAP_LO.wav
2,happy,CREMA,/kaggle/input/cremad/AudioWAV/1084_ITS_HAP_XX.wav
3,disgust,CREMA,/kaggle/input/cremad/AudioWAV/1067_IWW_DIS_XX.wav
4,disgust,CREMA,/kaggle/input/cremad/AudioWAV/1066_TIE_DIS_XX.wav


In [8]:
dir_list = os.listdir(RAV)
dir_list.sort()
dir_list.remove('audio_speech_actors_01-24')

emotion = []
gender = []
path = []
for i in dir_list:
    fname = os.listdir(RAV + i)
    for f in fname:
        part = f.split('.')[0].split('-')
        emotion.append(int(part[2]))
        temp = int(part[6])
        if temp%2 == 0:
            temp = "female"
        else:
            temp = "male"
        gender.append(temp)
        path.append(RAV + i + '/' + f)

        
RAV_df = pd.DataFrame(emotion)
RAV_df = RAV_df.replace({1:'neutral', 2:'neutral', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
RAV_df = pd.concat([pd.DataFrame(gender),RAV_df],axis=1)
RAV_df.columns = ['gender','emotion']
RAV_df['labels'] = RAV_df.emotion
RAV_df['source'] = 'RAVDESS'  
RAV_df = pd.concat([RAV_df,pd.DataFrame(path, columns = ['path'])],axis=1)
RAV_df = RAV_df.drop(['gender', 'emotion'], axis=1)
RAV_df.labels.value_counts()

neutral     288
happy       192
angry       192
sad         192
surprise    192
disgust     192
fear        192
Name: labels, dtype: int64

In [9]:
RAV_df.head(7)

,labels,source,path
0,neutral,RAVDESS,/kaggle/input/ravdess-emotional-speech-audio/A...
1,neutral,RAVDESS,/kaggle/input/ravdess-emotional-speech-audio/A...
2,sad,RAVDESS,/kaggle/input/ravdess-emotional-speech-audio/A...
3,neutral,RAVDESS,/kaggle/input/ravdess-emotional-speech-audio/A...
4,neutral,RAVDESS,/kaggle/input/ravdess-emotional-speech-audio/A...
5,fear,RAVDESS,/kaggle/input/ravdess-emotional-speech-audio/A...
6,happy,RAVDESS,/kaggle/input/ravdess-emotional-speech-audio/A...


In [10]:
dir_list = os.listdir(SAVEE)

emotion=[]
path = []
for i in dir_list:
    if i[-8:-6]=='_a':
        emotion.append('angry')
    elif i[-8:-6]=='_d':
        emotion.append('disgust')
    elif i[-8:-6]=='_f':
        emotion.append('fear')
    elif i[-8:-6]=='_h':
        emotion.append('happy')
    elif i[-8:-6]=='_n':
        emotion.append('neutral')
    elif i[-8:-6]=='sa':
        emotion.append('sad')
    elif i[-8:-6]=='su':
        emotion.append('surprise')
    else:
        emotion.append('error') 
    path.append(SAVEE + i)

# Now check out the label count distribution 
SAVEE_df = pd.DataFrame(emotion, columns = ['labels'])
SAVEE_df['source'] = 'SAVEE'
SAVEE_df = pd.concat([SAVEE_df, pd.DataFrame(path, columns = ['path'])], axis = 1)
SAVEE_df.labels.value_counts()

neutral     120
angry        60
surprise     60
disgust      60
happy        60
sad          60
fear         60
Name: labels, dtype: int64

In [11]:
SAVEE_df.head(7)

,labels,source,path
0,happy,SAVEE,/kaggle/input/surrey-audiovisual-expressed-emo...
1,fear,SAVEE,/kaggle/input/surrey-audiovisual-expressed-emo...
2,happy,SAVEE,/kaggle/input/surrey-audiovisual-expressed-emo...
3,disgust,SAVEE,/kaggle/input/surrey-audiovisual-expressed-emo...
4,angry,SAVEE,/kaggle/input/surrey-audiovisual-expressed-emo...
5,fear,SAVEE,/kaggle/input/surrey-audiovisual-expressed-emo...
6,happy,SAVEE,/kaggle/input/surrey-audiovisual-expressed-emo...


In [12]:
df = pd.concat([SAVEE_df, RAV_df, TESS_df], axis = 0)
print(df.labels.value_counts())
df.head()
df.to_csv("Data_path.csv",index=False)

neutral     808
angry       652
surprise    652
happy       652
sad         652
disgust     652
fear        652
Name: labels, dtype: int64


In [13]:
df['source'].value_counts()

TESS       2800
RAVDESS    1440
SAVEE       480
Name: source, dtype: int64

In [14]:
# Import our libraries
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import pandas as pd
import os
import IPython.display as ipd  # To play sound in the notebook

In [15]:
from time import sleep
from tqdm import tqdm

In [ ]:
def extract_features(data,sample_rate):
    result = np.array([])
    
    #spectral_bandwidth
    #spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=data, sr=sample_rate).T,axis=0)
    #result = np.hstack((result,spec_bw))
    
    #MFCC
    #mfcc = np.mean(librosa.feature.mfcc(y=data, 
     #                                   sr=sample_rate, 
      #                                  n_mfcc=13).T,axis=0)
    #result=np.hstack((result, mfcc))
    
   # zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    #result=np.hstack((result, zcr))
    
    #stft = np.abs(librosa.stft(data))
    #chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    #result = np.hstack((result, chroma_stft)) # stacking horizontally
    
   # rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    #result = np.hstack((result, rms)) # stacking horizontally
    
    #mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    #result = np.hstack((result, mel)) # stacking horizontally
    
    return result

In [ ]:
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def stretch(data, rate=0.7):
    return librosa.effects.time_stretch(data, rate)

def shift(data):
    shift_range = int(np.random.uniform(low=-10, high = 10)*1000)
    return np.roll(data, shift_range)

def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)


In [ ]:
def get_features(path):
    data, sample_rate = librosa.load(path, res_type='kaiser_fast',duration=2.5,sr=44100,offset=0.5)
    # without augmentation
    res1 = extract_features(data,sample_rate)
    result = np.array(res1)
    noise_data = noise(data)
    res2 = extract_features(noise_data,sample_rate)
    result = np.vstack((result, res2)) # stacking vertically
    #data with stretching and pitching
    new_data = stretch(data)
    data_stretch_pitch = pitch(new_data, sample_rate)
    res3 = extract_features(data_stretch_pitch,sample_rate)
    result = np.vstack((result, res3)) # stacking vertically
    return result

In [ ]:
X1, Y1 = [], []
for path, emotion in tqdm(zip(df.path, df.labels)):
    feature = get_features(path)
    
    for ele in feature:
        X1.append(ele)
        # appending emotion 3 times as we have made 3 augmentation techniques on each audio file.
        Y1.append(emotion)

In [ ]:
data, sample_rate = librosa.load(df.path.iloc[4000], res_type='kaiser_fast',duration=2.5,sr=44100,offset=0.5)
y = librosa.effects.harmonic(data)
tonnetz = np.mean(librosa.feature.tonnetz(y=y, sr=sample_rate).T,axis=0)
tonnetz.shape

In [ ]:
tonnetz=librosa.feature.tonnetz(y=y, sr=sample_rate).T
tonnetz.shape

In [ ]:
data, sample_rate = librosa.load(df.path.iloc[4000], res_type='kaiser_fast',duration=2.5,sr=44100,offset=0.5)
a = np.mean(librosa.feature.spectral_flatness(y=data).T,axis=0)
a.shape

In [ ]:
spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=data, sr=sample_rate).T,axis=0)
spec_bw.shape

In [ ]:
a

In [ ]:
data

In [ ]:
X11 = []
for i in range(len(X1)):
    X11.append(X1[i][0])

In [16]:
import pickle
open_file = open('/kaggle/input/without-ten/X11.pkl', "rb")
X = pickle.load(open_file)
open_file.close()

In [17]:
open_file = open('/kaggle/input/without-ten/Y11.pkl', "rb")
Y = pickle.load(open_file)
open_file.close()

In [18]:
len(X)

14160

In [19]:
type(X)

list

In [20]:
len(X[0])

157

In [21]:
Features = pd.DataFrame(X)
Features['labels'] = Y

In [22]:
Features=Features.fillna(0)

In [23]:
Features

,0,1,2,3,4,5,6,7,8,9,...,148,149,150,151,152,153,154,155,156,labels
0,1090.069388,-313.312164,151.610535,23.706287,26.743956,29.649649,10.801799,-3.875234,-24.508097,-13.172283,...,7.538793e-08,5.441303e-08,4.908057e-08,4.486316e-08,4.256390e-08,4.187426e-08,3.719218e-08,3.621465e-08,3.498884e-08,angry
1,5603.723522,-220.995168,86.997841,33.082131,24.466260,22.337011,8.877408,-4.881213,-16.141156,-10.881760,...,1.819575e-03,1.859388e-03,1.815507e-03,1.821846e-03,1.841691e-03,1.833580e-03,1.809761e-03,1.833716e-03,1.882496e-03,angry
2,1126.586315,-363.556061,137.863113,25.834318,28.230240,27.458530,6.845120,-8.739801,-24.722622,-11.041897,...,2.457574e-08,3.206178e-08,3.351928e-08,3.579163e-08,4.428324e-08,5.214509e-08,7.902064e-08,1.415829e-07,2.628523e-07,angry
3,1118.986365,-280.053314,164.992477,12.650513,0.411718,20.572817,12.793321,-3.151093,-20.558060,-7.187261,...,1.739802e-07,5.571374e-08,2.365663e-08,3.885421e-08,2.938769e-08,2.961217e-08,1.668467e-08,1.282859e-08,1.884284e-08,angry
4,6431.105833,-146.635954,74.388519,24.828422,5.553537,10.717650,8.924491,-3.810065,-11.656852,-4.308243,...,1.204099e-02,1.190575e-02,1.208748e-02,1.175759e-02,1.169913e-02,1.179324e-02,1.185429e-02,1.185520e-02,1.179482e-02,angry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14155,6054.731703,-349.915520,53.463040,2.785572,11.137381,0.646874,-5.267332,1.019780,-3.865511,-12.044798,...,1.721955e-04,1.765550e-04,1.746386e-04,1.801246e-04,1.697416e-04,1.719681e-04,1.715056e-04,1.691148e-04,1.688150e-04,surprise
14156,2505.193000,-501.696106,107.127228,-27.847162,39.521790,-11.526248,-2.342398,1.340641,-9.352740,-9.427408,...,2.415226e-11,2.166968e-11,2.139235e-11,1.988279e-11,2.876926e-11,5.516825e-11,3.028919e-10,7.196019e-11,9.407729e-11,surprise
14157,2252.144647,-467.417908,117.865967,-1.477675,65.936737,14.720181,8.500696,9.138531,-1.746968,-3.546331,...,1.951546e-07,1.819450e-07,1.704411e-07,1.610847e-07,1.538073e-07,1.463830e-07,1.437760e-07,1.380602e-07,1.355983e-07,sad
14158,6440.066801,-353.478041,47.496137,25.783479,29.346224,17.755978,6.971926,6.382404,1.053262,-5.089576,...,2.611206e-04,2.687480e-04,2.634849e-04,2.699890e-04,2.830480e-04,2.714880e-04,2.708660e-04,2.712532e-04,2.614771e-04,sad


In [24]:
import keras
from keras import regularizers
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Embedding, LSTM
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint

# sklearn
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Other  
import librosa
import librosa.display
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import pandas as pd
import seaborn as sns
import glob 
import os
import pickle
import IPython.display as ipd

In [25]:
X_train, X_test, y_train, y_test = train_test_split(Features.drop(columns=['labels']),Features.labels, test_size=0.25, shuffle=True, random_state=42)

In [26]:
# Lts do data normalization 
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std

In [27]:
# Lets few preparation steps to get it into the correct format for Keras 
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# one hot encode the target 
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

print(X_train.shape)
print(lb.classes_)

(10620, 157)
['angry' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']


In [28]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
X_train.shape

(10620, 157, 1)

In [41]:
model = Sequential()
model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))  # X_train.shape[1] = No. of Columns
model.add(Activation('relu'))
model.add(Conv1D(256, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(128, 8, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 8, padding='same'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(7)) # Target class nu
model.add(Activation('softmax'))
# opt = keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)
opt = keras.optimizers.Adam(lr=0.00001)
#opt = keras.optimizers.RMSprop(lr=0.00001, decay=1e-6)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_32 (Conv1D)           (None, 157, 256)          2304      
_________________________________________________________________
activation_36 (Activation)   (None, 157, 256)          0         
_________________________________________________________________
conv1d_33 (Conv1D)           (None, 157, 256)          524544    
_________________________________________________________________
batch_normalization_8 (Batch (None, 157, 256)          1024      
_________________________________________________________________
activation_37 (Activation)   (None, 157, 256)          0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 157, 256)          0         
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 19, 256)          

In [42]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=9,restore_best_weights=True)
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
model_history=model.fit(X_train, y_train, batch_size=2, epochs=200, validation_data=(X_test, y_test),callbacks=[callback])

Epoch 1/200
5310/5310 [==============================] - 31s 6ms/step - loss: 1.7078 - accuracy: 0.3547 - val_loss: 1.3045 - val_accuracy: 0.5986
Epoch 2/200
5310/5310 [==============================] - 30s 6ms/step - loss: 1.1447 - accuracy: 0.6084 - val_loss: 1.0880 - val_accuracy: 0.6607
Epoch 3/200
5310/5310 [==============================] - 30s 6ms/step - loss: 0.9343 - accuracy: 0.6751 - val_loss: 0.9473 - val_accuracy: 0.7169
Epoch 4/200
5310/5310 [==============================] - 30s 6ms/step - loss: 0.8325 - accuracy: 0.7121 - val_loss: 0.8371 - val_accuracy: 0.7212
Epoch 5/200
5310/5310 [==============================] - 30s 6ms/step - loss: 0.7595 - accuracy: 0.7301 - val_loss: 0.7837 - val_accuracy: 0.7424
Epoch 6/200
5310/5310 [==============================] - 30s 6ms/step - loss: 0.6963 - accuracy: 0.7493 - val_loss: 0.7368 - val_accuracy: 0.7551
Epoch 7/200
5310/5310 [==============================] - 30s 6ms/step - loss: 0.6585 - accuracy: 0.7641 - val_loss: 0.6874 -

In [ ]:
X_train

In [ ]:
Y